In [1]:
import numpy as np

# pretty print numpy matrices
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})

In [2]:
# the DH paramters for KUKA robot as per written document
# alpha, a, d, theta (adjustmet)
# the angles we will use to control the robot will be added to the theta paramters
DH = [[0       , 0     , 0.75 , 0        ],
      [-np.pi/2, 0.35  , 0    , - np.pi/2],
      [0       , 1.25  , 0    , 0        ],
      [-np.pi/2, -0.054, 1.5  , 0        ],
      [np.pi/2 , 0     , 0    , 0        ],
      [-np.pi/2, 0     , 0    , 0        ],
      [0       , 0     , 0.303, 0        ]]

In [3]:
# a help function that builds a transformation matrix given the 4 DH paramters for that joint
def LinkTransform(params):
    # params is a list of length 4: alpha, a, d, theta in this order
    alpha, a, d, theta = params

    # returns a Matrix of transformation for the given DH paramters
    return np.array([[np.cos(theta)              , -np.sin(theta)             , 0             , a],
                     [np.sin(theta)*np.cos(alpha), np.cos(theta)*np.cos(alpha), -np.sin(alpha), -np.sin(alpha)*d],
                     [np.sin(theta)*np.sin(alpha), np.cos(theta)*np.sin(alpha), np.cos(alpha) , np.cos(alpha)*d],
                     [0                          , 0                          , 0             , 1]])

In [4]:
# builds the transformation matrices based on the DH and the theta paramters
# it resurns the final transformation matrix
def ChainLinkTransform(DH, thetas, printTransformations = False):
    for i in range(len(DH)):
        params = list(DH[i])    # we need to make a copy 
        params[3] += thetas[i]
        T = LinkTransform(params)
        if i == 0:
            result = T
        else:
            result = np.matmul(result, T)

        if printTransformations:
            print("T%d_%d = " % (i, i+1))
            print(T)
            print("T0_%d = " % (i+1))
            print(result)
            
    return result

In [5]:
# extracts the position and orientation from the quaternion matrix
def OrientationFromHom(hom):
    pos = hom[:,3].T
    orient = np.array([np.arctan2(hom[2,1], hom[2,2]),
                       np.arctan2(-hom[2,0], np.sqrt(hom[0,0]**2 + hom[1,0]**2)),
                       np.arctan2(hom[1,0], hom[0,0])])
    
    return pos[0:3], orient

In [6]:
# builds a transformation matrix for the orientation adjustment of the gripper
# so that we are consistent with the URDF representaiton of the gripper
def GripperAdust(r_z = np.pi, r_y = -np.pi/2):
    R_z = np.array([[np.cos(r_z), -np.sin(r_z), 0, 0],
                    [np.sin(r_z), np.cos(r_z) , 0, 0],
                    [0          , 0           , 1, 0],
                    [0          , 0           , 0, 1]])
    R_y = np.array([[np.cos(r_y) , 0          , np.sin(r_y), 0],
                    [0           , 1          , 0          , 0],
                    [-np.sin(r_y), 0          , np.cos(r_y), 0],
                    [0           , 0          , 0          , 1]])
    return np.matmul(R_z, R_y)

In [7]:
def CalculateEffector(DH, thetas, printTransformations = False):
    res = ChainLinkTransform(DH, thetas, printTransformations)
    # adjust orientation
    adj = GripperAdust()
    res = np.matmul(res, adj)
    if printTransformations:
        print("Adjustment= ")
        print(adj)
        print("Adjusted  = ")
        print(res)
     
    pos, orient = OrientationFromHom(res)
    print("pos    = "+str(pos))
    print("orient = "+str(orient))   

In [8]:
th = [0, 0, 0, 0, 0, 0, 0]
CalculateEffector(DH, th)

pos    = [ 2.15300  0.00000  1.94600]
orient = [-0.00000  0.00000  0.00000]


ROS Reported:<br>
`Trans = [2.153, 0.000, 1.947]
Rot = [0.000, 0.000, 0.000]`

In [9]:
th = [0.99, 0, 0, 0, 0, 0, 0]
CalculateEffector(DH, th)

pos    = [ 1.18133  1.79996  1.94600]
orient = [-0.00000  0.00000  0.99000]


ROS Reported:<br>
`Trans = [1.173, 1.805, 1.947]
Rot = [0.000, 0.000, 0.994]`

In [10]:
th = [0.99, 0.32, 0, 0, 0, 0, 0]
CalculateEffector(DH, th)

pos    = [ 1.33754  2.03797  1.31812]
orient = [-0.00000  0.32000  0.99000]


ROS Reported:<br>
`Trans = [1.328, 2.044, 1.321]
Rot = [0.000, 0.319, 0.994]`

In [11]:
th = [0.99, 0.32, -0.49, 0, 0, 0, 0]
CalculateEffector(DH, th)

pos    = [ 1.38783  2.11461  2.18836]
orient = [-0.00000 -0.17000  0.99000]


ROS Reported:<br>
`Trans = [1.377, 2.120, 2.190]
Rot = [0.000, -0.171, 0.994]`

In [12]:
th = [0.99, 0.32, -0.49, 1.05, 0, 0, 0]
CalculateEffector(DH, th)

pos    = [ 1.38783  2.11461  2.18836]
orient = [ 1.05000 -0.17000  0.99000]


ROS Reported:<br>
`Trans = [1.377, 2.120, 2.190]
Rot   = [1.046, -0.171, 0.994]`

In [13]:
th = [0.99, 0.32, -0.49, 1.05, 0.99, 0, 0]
CalculateEffector(DH, th)

pos    = [ 1.14188  2.14032  2.04100]
orient = [ 1.12313  0.32273  1.86052]


ROS Reported:<br>
`Trans = [1.133, 2.144, 2.042]
Rot   = [1.119, 0.324, 1.860]`

In [14]:
th = [0.99, 0.32, -0.49, 1.05, 0.99, -0.44, 0]
CalculateEffector(DH, th)

pos    = [ 1.14188  2.14032  2.04100]
orient = [ 0.68313  0.32273  1.86052]


ROS Reported:<br>
`Trans = [1.133, 2.144, 2.042]
Rot   = [0.678, 0.324, 1.860]`